In [1]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [3]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Access Token = a3eff472316a350ec1039ef3c34222de023dc0e4



In [12]:
activities = pd.json_normalize(my_dataset)

In [14]:
activities.columns 

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'average_heartrate', 'max_heartrate',
       'suffer_score', 'average_cadence'],
      dtype='object')

In [15]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date
activities.head(5)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_heartrate,max_heartrate,suffer_score,average_cadence,start_time
0,2,Chandler Lane x2,2090.5,643,658,28.6,Run,Run,0.0,8290856067,...,43537797,1,a8290856067,wkr`FjiofVFURa@NSRO^KNDVIN?XUTIVYVSRGNMNGh@k@N...,2,NaN,NaN,NaN,NaN,13:23:05
1,2,Chandler Lane Run,3373.0,1057,1065,55.6,Run,Run,0.0,8287871741,...,43537797,1,a8287871741,obr`FdbofVPIRWLGLCR@l@C`@OXWd@OVAN@XKH?^Sl@STS...,2,NaN,NaN,NaN,NaN,15:02:07
2,2,SacYard run,4513.9,2017,2177,4.5,Run,Run,0.0,8273019317,...,43537797,1,a8273019317,wskjFdqjdVICMMQIKSDMRKJUBSNS@INK@I?ADEEKYe@US]...,2,NaN,NaN,NaN,NaN,18:07:34
3,2,Lunch walk with Piper,3057.6,1635,2787,16.5,Walk,Walk,NaN,8225071724,...,43537797,1,a8225071724,oxjjF`xqdV]I{@[WAOOAYR{@LH\?f@HBB@HVFNTLIB@?DC...,2,NaN,NaN,NaN,NaN,12:31:32
4,2,Night run with Piper,2422.8,920,1151,2.2,Run,Run,0.0,8222096181,...,43537797,1,a8222096181,yuijFdwrdVLBf@Zd@NxAp@NDHHrAXh@@`@HX@PNH@FDE\G...,2,NaN,NaN,NaN,NaN,22:05:51


In [23]:
# import modules
import os
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm

In [24]:
# add decoded summary polylines
activities['map.polyline'] = activities['map.summary_polyline'].apply(polyline.decode)
print("turkey")
#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities.index):
    activity = activities.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

turkey


  0%|          | 0/94 [00:00<?, ?it/s]

In [ ]:
# add elevation data to dataframe
activities['map.elevation'] = elevation_data

In [ ]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [ ]:
# filter down to Ride, Run, and Swim activities
activities_filtered = activities_copy.query("type == 'Ride' | type == 'Run' | type == 'Swim'")
print(len(activities_filtered))